<a href="https://colab.research.google.com/github/GiriSriramula/GiriSriramula/blob/main/kerastuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner


     |████████████████████████████████| 133 kB 5.4 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labes),(test_images,test_labels)= fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255
test_images = test_images/255

In [ ]:
train_images[0].shape


(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D( 
                                filters = hp.Int('conv_1_filter',min_value =32 , max_value =128, step=16),
                                kernel_size = hp.Choice('conv_1_kernal',values=[3,5]),
                                activation ='relu',
                                input_shape =(28,28,1)
  ),
  keras.layers.Conv2D(
      filters = hp.Int('conv_1_filter',min_value =32 , max_value =128, step=16),
      kernel_size = hp.Choice('conv_1_kernal',values=[3,5]),
      activation ='relu'
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(
      units=hp.Int('dense_1_units',min_value =32, max_value=128,step=16),
      activation ='relu'
  ),
  keras.layers.Dense(10,activation ='softmax') 
  ])
  model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials =5,
                            directory='output',
                            project_name ="Mnistfashion")

In [ ]:
tuner_search.search(train_images,train_labes,epochs=3, validation_split =0.1)

Trial 3 Complete [00h 11m 22s]
val_accuracy: 0.8604999780654907

Best val_accuracy So Far: 0.9103333353996277
Total elapsed time: 00h 40m 08s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
96                |64                |conv_1_filter
5                 |3                 |conv_1_kernal
112               |96                |dense_1_units
0.001             |0.001             |learning_rate

Epoch 1/3
1688/1688 [==============================] - 658s 390ms/step - loss: 0.4121 - accuracy: 0.8494 - val_loss: 0.3005 - val_accuracy: 0.8893
Epoch 2/3
1414/1688 [========================>.....] - ETA: 1:43 - loss: 0.2682 - accuracy: 0.9010

store the best model into mdoel



In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 96)                3539040   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,577,578
Trainable params: 3,577,578
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labes,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1123 - accuracy: 0.9578 - val_loss: 0.2777 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0747 - accuracy: 0.9723 - val_loss: 0.3010 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0502 - accuracy: 0.9814 - val_loss: 0.3592 - val_accuracy: 0.9148
Epoch 7/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0357 - accuracy: 0.9872 - val_loss: 0.3792 - val_accuracy: 0.9183
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0259 - accuracy: 0.9907 - val_loss: 0.4249 - val_accuracy: 0.9167
Epoch 9/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0218 - accuracy: 0.9923 - val_loss: 0.4654 - val_accuracy: 0.9187
Epoch 10/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0190 - accuracy: 0.9935 - val_loss: 0.4992 